In [1]:
def solution(total_lambs): 
    
    #Function that calculates the next henchman LAMBs if being stingy
    def stingy(hench_list):
        
        return(sum(hench_list[len(hench_list)-2:])) #Next henchman must have LAMBS = two henchmen below
    
    #Function that calculates the next henchman LAMBs if being generous
    def generous(hench_list):
        return(2*(hench_list[len(hench_list)-1])) #Next henchman must have 2x that of 1 henchman below

    #for loop inputs
    func_list = [stingy, generous] #iterate through each function
    compare_list = list(range(2)) #empty list
    
    #iterate through each function 
    for e in range(2):
        hench_list = [1] #initialise list - first henchman always has 1 LAMB
        leftover = total_lambs #how many LAMBS are leftover
        for i in range(total_lambs): #loop through each LAMB
            new_value = func_list[e](hench_list) #calculate LAMBS for next henchman
            
            if leftover >= new_value and total_lambs!=1: #if there are LAMBS leftover
                hench_list.append(new_value) #add next henchman LAMBS to list
                leftover = total_lambs - sum(hench_list) #recalculate leftovers

            else: #if not LAMBS leftover
                #compare_list[e] = len(hench_list)
                compare_list[e] = hench_list
                #print(str(e))
    
    #Calculate difference 
    return(compare_list)
    #return(compare_list[0] - compare_list[1])

In [ ]:
for i in range(72028,1000000):
    sol = solution(i)
    for e in range(len(sol)):
        if e == 0: name = 'stingy'
        if e == 1: name = 'generous'
        for t in range(len(sol[e])-1):
            if sol[e][t+1] > 2*sol[e][t]:
                print('error 2  ' + str(i) + ' ' + name)
            if sol[e][t+1] < sum(sol[e][t-1:t+1]):
                print('error 3' + str(i) + ' ' + name)

In [3]:
i

72027